# 1. Import Data



In [1]:
#Import Dependencies 
from nilearn import plotting
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nilearn import image
import nibabel as nib
import os
from nilearn.connectome import ConnectivityMeasure
from nilearn.input_data import NiftiLabelsMasker
from brainspace.plotting import plot_hemispheres
from brainspace.datasets import load_parcellation, load_conte69
from brainspace.gradient import GradientMaps
labeling = load_parcellation('schaefer', scale=100, join=True)
# and load the conte69 surfaces
surf_lh, surf_rh = load_conte69()

/Users/mioulin/opt/anaconda3/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
# Load Schefer atlas for parcellation # 100 ROI, 7Networks
from nilearn import plotting, datasets
atlas_file = datasets.fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7,resolution_mm=2)
atlas_filename = atlas_file.maps
labels = atlas_file.labels

In [3]:
# Import Masker 
from nilearn.input_data import NiftiLabelsMasker
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                           memory='nilearn_cache', verbose=5)
masker

NiftiLabelsMasker(labels_img='/Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz',
                  memory='nilearn_cache', standardize=True, verbose=5)

# 2. Import Gradient

In [4]:
# Load hcp functional connectivitty from brainspace toolbox: 
from brainspace.datasets import load_group_fc, load_parcellation, load_conte69

# First load mean connectivity matrix and Schaefer parcellation
conn_matrix = load_group_fc('schaefer', scale=100)
labeling = load_parcellation('schaefer', scale=100, join=True)
from brainspace.gradient import GradientMaps

# Ask for one fc hcp grad
gm = GradientMaps(n_components=3, random_state=0)
gm.fit(conn_matrix)
grad_hcp = gm.gradients_


/Users/mioulin/opt/anaconda3/lib/python3.9/site-packages/brainspace/gradient/embedding.py:70: UserWarning: Affinity is not symmetric. Making symmetric.
  warnings.warn('Affinity is not symmetric. Making symmetric.')


# 3. DMT Dataset time-series Extraction

In [7]:
import nibabel as nib
atlas_file = nib.load('/Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz')

with open('/Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order.txt', 'r') as f:
    labels = f.read().split('\n')
masker = NiftiLabelsMasker(labels_img=atlas_file, standardize=True, verbose=1,
                           memory="nilearn_cache", memory_level=2)
labels = [x.split('\t')[1][10:] for x in labels]

In [6]:
#Function for time series for each dataset
def time_series_all(file_name):
    time_series = masker.fit_transform(file_name)
    return time_series

In [7]:
#Extract time-series for each subject in each dataset
all_time_series = dict()
all_time_series['dmt']= dict()
all_time_series['dmt_pcb']= dict()

os.chdir('/Users/mioulin/Desktop/data/pcb')
rest_files_pcb = [ x for x in os.listdir('.') if x . endswith('.nii')]
for file in rest_files_pcb:
    all_time_series['dmt_pcb'][file[51:53]] = time_series_all(file)
    
os.chdir('/Users/mioulin/Desktop/data/dmt')
rest_files_dmt = [ x for x in os.listdir('.') if x . endswith('.nii')]
for file in rest_files_dmt:
    all_time_series['dmt'][file[51:53]] = time_series_all(file)

[NiftiLabelsMasker.fit_transform] loading data from /Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
[Memory]0.0s, 0.0min    : Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from /Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
[Memory]0.1s, 0.0min    : Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from /Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
[Memory]0.1s, 0.0min    : Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from /Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Netwo

# 4. Correlate Dmt Time-Series with Gradient 

In [9]:

import numpy as np

corr_results_dmt = {}
b = grad_hcp.T

for state, subjects in all_time_series.items():
    corr_results_dmt[state] = {}
    for subject, series in  all_time_series[state].items():
        X = np.array(series)  # Convert the series data to a NumPy array
        corr_results_dmt[state][subject] = np.corrcoef(X, b)

In [10]:
# Save it
import pickle
with open('corr_results_dmt.pkl', 'wb') as f:
    pickle.dump(corr_results_dmt, f)